In [1]:
import pandas as pd
import numpy as np
from basketball_dataset.nba_tracking_data_15_16 import NbaTracking
from basketball_dataset.dataset_operations import *

c:\Users\zcoch\Triton Ball Workshops\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = NbaTracking(config_name="medium")
dataset.download_and_prepare("basketball_dataset_full/data")
nba_dataset = dataset.as_dataset()
filtered_events = filter_candidate_events(nba_dataset["train"])
turnovers_and_shots = pd.DataFrame(filtered_events)

Generating train split: 46006 examples [21:46, 35.21 examples/s] 


In [10]:
turnovers_and_shots[["gameid","gamedate","event_info","primary_info","secondary_info","visitor","home"]].to_csv("turnovers_and_shots.csv",index=False)

In [19]:
events = pd.json_normalize(turnovers_and_shots["event_info"])
events = pd.concat([events,turnovers_and_shots["primary_info"].apply(lambda prim_info: prim_info["team_id"])],axis=1).rename({"primary_info":"event_team"},axis=1)
turnovers = events[events["event_type"]=="turnover"]
for i, turnover in turnovers.iterrows():
    game_id = turnover["game_id"]
    quarter = turnover["quarter"]
    game_clock = turnover["game_clock"]
    shot_after = events[(events["game_id"]==game_id) & (events["quarter"]==quarter) & (events["event_type"]=="made shot") & (events["game_clock"]<game_clock) & (events["game_clock"]>=game_clock-10)] # Shot within 10 seconds of turnover
    if len(shot_after) >= 1:
        turnovers.loc[i,"made_shot_after"] = True
    else:
        turnovers.loc[i,"made_shot_after"] = False

C:\Users\zcoch\AppData\Local\Temp\ipykernel_54652\576754249.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  turnovers.loc[i,"made_shot_after"] = False


In [21]:
turnovers.to_csv("turnovers.csv",index=False)